<h1>
<hr style=" border:none; height:3px;">
<center>Benchmark pipeline</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE</center></h4>

# Introduction
<div style="background-color: #e3f2fd; padding: 15px; border-radius: 8px; border: 1px solid #81d4fa; display: flex; flex-direction: column; align-items: flex-start; max-width: 1125px;">
  <div style="display: flex; align-items: center; margin-bottom: 10px;">
    <img src="https://img.icons8.com/ios-filled/50/000000/info.png" alt="info" style="margin-right: 15px; width: 30px; height: 30px;">
    <span style="font-size: 16px; font-weight: bold; color: #01579b;">Information:</span>
  </div>
  <p style="font-size: 14px; color: #01579b;">
    After generating two batches of 8 images with four different comfyUI workflows, the database was annotated by humans to score three values: <br>
    - Logo Boolean Score (Yes/No)<br>
    - Aesthetic score (0-10)<br>
    - Business Score (0-10).<br><br>
    Thereafter, the goal is to compare those scores and see if the human feedback is correlated with two additional scores - <b>CLIP score</b> to calculate similarity between prompt and image, or <b>LLM custom scores</b> (with GPT-4o as annotator) detailed in scientific papers - see notes<br>
    For the record, the database was <b>NOT</b> annotated by myself, who generated the logo to avoid any bias.
  </p>
</div>
